In [1]:
import pandas as pd
import numpy as np

In [5]:
splits = pd.read_csv("/home/kwatchar3/projects/query-bandit/reproducibility/splits.csv")

In [8]:
test_files = splits[splits.split == 5]["song_id"].tolist()

In [96]:
BANDIT_RESULTS = "/home/kwatchar3/projects/query-bandit/results/vdbo/bandit4.csv"
DEMUCS_RESULTS = "/home/kwatchar3/projects/query-bandit/results/vdbo/htdemucs4.csv"
SPLEETER_RESULTS = "/home/kwatchar3/projects/query-bandit/results/vdbo/spleeter4.csv"

ORACLE_RESULTS = "/home/kwatchar3/projects/query-bandit/results/vdbo/oracle4.csv"

DEMUCS6_RESULTS = "/home/kwatchar3/projects/query-bandit/results/vdbgpo/htdemucs6.csv"
SPLEETER5_RESULTS = "/home/kwatchar3/projects/query-bandit/results/vdbgpo/spleeter5.csv"

ORACLE6_RESULTS = "/home/kwatchar3/projects/query-bandit/results/vdbgpo/oracle6.csv"

In [82]:
stem_dtype = pd.CategoricalDtype(["vocals", "drums", "bass", "guitar", "piano", "other"], ordered=True)

In [68]:
def stats_from_moisesdb(path, test_files):
    df = pd.read_csv(path)

    if "song" in df.columns:
        df = df.rename(columns={"song": "track_id"})

    if "sdr" in df.columns:
        df = df.rename(columns={"sdr": "snr"})

    df.loc[df.stem == "vdbo_others", "stem"] = "other"

    df.stem = df.stem.astype(stem_dtype)

    df = df[df['track_id'].isin(test_files)]

    if "model" in df.columns:
        cols = ["model", "stem"]
    else:
        cols = ["stem"]

    df = df.pivot_table(index=["track_id"], columns=cols, values=["snr"])

    return df.describe()

In [69]:
stats_from_moisesdb(DEMUCS_RESULTS, test_files).round(1).loc["50%"]

     stem  
snr  vocals     9.1
     drums     11.0
     bass      12.2
     other      7.3
Name: 50%, dtype: float64

In [70]:
stats_from_moisesdb(SPLEETER_RESULTS, test_files).round(1).loc["50%"]

     stem  
snr  vocals    7.4
     drums     6.6
     bass      6.8
     other     5.0
Name: 50%, dtype: float64

In [71]:
stats_from_moisesdb(BANDIT_RESULTS, test_files).round(1).loc["50%"]

     stem  
snr  vocals     9.1
     drums      9.9
     bass      10.6
     other      6.4
Name: 50%, dtype: float64

In [72]:
stats_from_moisesdb(ORACLE_RESULTS, test_files).round(1).loc["50%"]

     model  stem  
snr  IBM    vocals     8.6
            drums      7.6
            bass       6.7
            other      5.7
     IRM    vocals    10.3
            drums      9.2
            bass       8.8
            other      7.6
     MWF    vocals    10.2
            drums      9.0
            bass       9.1
            other      7.8
     cIRM   vocals     9.7
            drums      8.5
            bass       7.4
            other      6.5
Name: 50%, dtype: float64

In [91]:
stats_from_moisesdb(DEMUCS6_RESULTS, test_files).round(1).loc[["50%"]].T

50%
    stem        
snr vocals   8.9
    drums   11.6
    bass    12.4
    guitar   2.4
    piano    1.7
    other    0.8

In [97]:
stats_from_moisesdb(ORACLE6_RESULTS, test_files).round(1).loc[["50%"]].T

50%
    model stem        
snr IBM   vocals   8.4
          drums    8.1
          bass     6.0
          guitar   3.3
          piano    2.8
          other    3.2
    IRM   vocals  10.0
          drums    9.6
          bass     7.8
          guitar   5.2
          piano    5.0
          other    5.1
    MWF   vocals   9.9
          drums    9.5
          bass     8.3
          guitar   5.1
          piano    4.8
          other    4.9
    cIRM  vocals   9.6
          drums    9.1
          bass     6.7
          guitar   3.8
          piano    3.3
          other    3.5

In [122]:
oracle = pd.read_csv(ORACLE_RESULTS)["track_id"].unique()

splits["avail"] = splits["song_id"].isin(oracle)

missing = splits[(splits['split'] == 5) & (~splits["avail"])].song_id

In [115]:
data_root = "/home/kwatchar3/Documents/data/moisesdb/canonical"

In [113]:
import os, json

In [123]:
for s in missing:
    with open(os.path.join(data_root, s, "data.json")) as f:
        info = json.load(f)

        print(info['song'], "by", info['artist'])

What Are The Odds by Frank O Pinions
Gloria's Swan Song by Firefly
